In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.init as init
import random

datasets.MNIST('/mnt/hgfs/E/DATASETS/',train=True,download=True,
                       transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                       ]))

Dataset MNIST
    Number of datapoints: 60000
    Root location: /mnt/hgfs/E/DATASETS/
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [2]:
import subprocess

result = subprocess.run(['build/dcgan'], capture_output=True, text=True)

output = result.stdout

lines = output.split('\n')
parsing = False
cpp_array_elapsed_time = []
cpp_total=[0,0,0]
for line in lines:
    if line.startswith('========'):
        parsing = True
    elif parsing:
        if line.startswith('STEP'):
            _, _, time = line.split(':')
            cpp_array_elapsed_time.append(int(time))
        elif line.startswith('Total Time'):
            cpp_total[0] = int(line.split(':')[1])
        elif line.startswith('Sum Time'):
            cpp_total[1] = int(line.split(':')[1])
        elif line.startswith('Step Count'):
            cpp_total[2] = int(line.split(':')[1])

print(output)
print(cpp_array_elapsed_time)
print(cpp_total)

KeyboardInterrupt: 

# PYTHON

## Imports

In [9]:
# Load in relevant libraries, and alias where appropriate
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

## Global Variables

In [10]:
batch_size = 64
num_classes = 10
learning_rate = 0.001
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Dataloader

In [11]:
train_dataset = torchvision.datasets.MNIST(root = 'E:/DATASETS/',
                                           train = True,
                                           transform = transforms.Compose([
                                               transforms.Resize((32,32)),
                                               transforms.ToTensor(),
                                               transforms.Normalize(mean = (0.1307,), std = (0.3081,))]),
                                           download = True)
train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                           batch_size = batch_size,
                                           shuffle = True)


## Model

In [12]:
#Defining the convolutional neural network
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(6),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(400, 120)
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(120, 84)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(84, num_classes)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        out = self.relu(out)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        return out

## Definitions

In [13]:
model = LeNet5(num_classes).to(device)

#Setting the loss function
cost = nn.CrossEntropyLoss()

#Setting the optimizer with the model parameters and learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#this is defined to print how many steps are remaining when training
total_step = len(train_loader)

## Train

In [14]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        #Forward pass
        outputs = model(images)
        loss = cost(outputs, labels)
        #Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 400 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/10], Step [400/938], Loss: 0.1547
Epoch [1/10], Step [800/938], Loss: 0.1189
Epoch [2/10], Step [400/938], Loss: 0.0468
Epoch [2/10], Step [800/938], Loss: 0.0062
Epoch [3/10], Step [400/938], Loss: 0.0180
Epoch [3/10], Step [800/938], Loss: 0.0014
Epoch [4/10], Step [400/938], Loss: 0.0429
Epoch [4/10], Step [800/938], Loss: 0.0163
Epoch [5/10], Step [400/938], Loss: 0.0100
Epoch [5/10], Step [800/938], Loss: 0.0158
Epoch [6/10], Step [400/938], Loss: 0.0005
Epoch [6/10], Step [800/938], Loss: 0.0074
Epoch [7/10], Step [400/938], Loss: 0.0007
Epoch [7/10], Step [800/938], Loss: 0.0034
Epoch [8/10], Step [400/938], Loss: 0.0053
Epoch [8/10], Step [800/938], Loss: 0.0145
Epoch [9/10], Step [400/938], Loss: 0.0023
Epoch [9/10], Step [800/938], Loss: 0.0033
Epoch [10/10], Step [400/938], Loss: 0.0092
Epoch [10/10], Step [800/938], Loss: 0.0154
